In [2]:
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
import re
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [3]:
from dk_weekly_update import DkWeeklyUpdate as dk

In [4]:
df = pd.read_csv('input/rb/rb_loc_cleaned.csv')
df['year'].unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype=int64)

In [5]:
df[df['Name']=='Jordan Howard']

,games,year,Name,Team,Week,Opponent,att,yds,td,tar,rec_yds,def_rk,h/a_1,surface_0,points
11822,1,2018,Jordan Howard,CHI,1,GB,15,82,0,5,25,11,0.0,1.0,15.7
11937,2,2018,Jordan Howard,CHI,2,SEA,14,35,0,4,33,16,1.0,1.0,9.8
12050,3,2018,Jordan Howard,CHI,3,ARI,24,61,1,2,20,31,0.0,1.0,16.1
12155,4,2018,Jordan Howard,CHI,4,TB,11,25,0,1,0,26,1.0,1.0,2.5
12367,5,2018,Jordan Howard,CHI,6,MIA,14,69,0,1,0,28,0.0,1.0,5.9
12443,6,2018,Jordan Howard,CHI,7,NE,12,39,1,1,9,20,1.0,1.0,11.8
12540,7,2018,Jordan Howard,CHI,8,NYJ,22,81,1,1,0,21,1.0,1.0,14.1


In [4]:
def_2018 = pd.read_csv('input/def/fantasy-football-points-allowed-defense-by-position.csv')
def_2018.head()


,Rank,Name,Team,Position,Week,Opponent,DraftKingsQuarterbackFantasyPointsAllowedAverage,DraftKingsRunningbackFantasyPointsAllowedAverage,DraftKingsWideReceiverFantasyPointsAllowedAverage,DraftKingsTightEndFantasyPointsAllowedAverage,DraftKingsKickerFantasyPointsAllowedAverage,DraftKingsFantasyPointsAllowedAverage
0,1,Chicago Bears,CHI,DST,8,NYJ,12.32,10.0,20.2,8.6,5,56.12
1,2,Arizona Cardinals,ARI,DST,8,SF,12.50,9.8,28.3,10.7,7,68.30
2,3,New England Patriots,NE,DST,8,BUF,10.42,22.0,29.0,9.0,9,79.42
3,4,Pittsburgh Steelers,PIT,DST,8,CLE,15.10,16.5,29.6,12.4,7,80.60
4,5,Minnesota Vikings,MIN,DST,8,NO,10.86,38.8,17.9,0.0,16,83.56


In [18]:
df.sort_values(['year','Week'],inplace=True)
df.tail()

,games,year,Name,Opponent,Team,Week,att,yds,td,tar,rec_yds,def_rk,h/a_1,surface_0,points
12395,12,2018,Ty Montgomery,SF,GB,6,4,12,1,3,10,22,0.0,1.0,10.2
12396,6,2018,Tyler Ervin,BUF,HOU,6,0,0,0,0,0,18,1.0,0.0,0.0
12397,6,2018,Wayne Gallman,PHI,NYG,6,4,17,0,5,25,8,1.0,0.0,6.2
12398,6,2018,Wendell Smallwood,NYG,PHI,6,18,51,0,2,0,19,0.0,0.0,6.1
12420,22,2018,Kareem Hunt,NE,KC,6,10,80,0,6,105,21,0.0,0.0,32.5


In [6]:
wk8 = pd.read_csv('input/rb/rb_wk8.csv')

In [7]:
wk8.head()

,Rank,Name,Team,Position,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings
0,1,James Conner,PIT,RB,8,CLE,24,146,6.1,2,6,5,66,0,1,0,41.2,41.2
1,2,Todd Gurley,LAR,RB,8,GB,25,114,4.6,0,7,6,81,1,0,0,36.5,36.5
2,3,Marlon Mack,IND,RB,8,OAK,25,132,5.3,2,4,2,17,0,0,0,31.9,31.9
3,4,Joe Mixon,CIN,RB,8,TB,21,123,5.9,2,4,3,15,0,0,0,31.8,31.8
4,5,Adrian Peterson,WAS,RB,8,NYG,26,149,5.7,1,1,1,7,1,1,1,30.6,30.6


In [8]:
location = pd.read_csv('input/location.csv')
location.tail()

,Unnamed: 0,date,day,week,home,away,year
1653,116,"October 28, 2018",Sun,8,OAK,IND,2018
1654,117,"October 28, 2018",Sun,8,LAR,GB,2018
1655,118,"October 28, 2018",Sun,8,ARI,SF,2018
1656,119,"October 28, 2018",Sun,8,MIN,NO,2018
1657,120,"October 29, 2018",Mon,8,BUF,NE,2018


In [9]:
position_dict = {'qb':'DraftKingsQuarterbackFantasyPointsAllowedAverage',
                    'wr':'DraftKingsWideReceiverFantasyPointsAllowedAverage',
                    'rb':'DraftKingsRunningbackFantasyPointsAllowedAverage',
                    'te':'DraftKingsTightEndFantasyPointsAllowedAverage'}

team_dict = {'New England Patriots':'NE',
            'Green Bay Packers':'GB',
            'Tampa Bay Buccaneers':'TB',
            'Kansas City Chiefs':'KC',
            'San Diego Chargers':'SD',
            'St. Louis Rams':'STL',
            'New Orleans Saints':'NO',
            'New York Jets':'NYJ',
            'New York Giants':'NYG',
            'San Francisco 49ers':'SF',
            'Los Angeles Rams':'LAR',
            'Jacksonville Jaguars':'JAX',
            'Los Angeles Chargers':'LAC'}

In [135]:
rb_dict = {}
for player in df['Name'].unique():
    rb_dict[player] = df[df['Name']==player]
    rb_dict[player].sort_values(['year','Week'], inplace=True)
    rb_dict[player].reset_index(drop=True, inplace=True)
    rb_dict[player].reset_index(inplace=True)
    rb_dict[player].rename(columns={'index':'games'}, inplace=True)
    rb_dict[player]['games' ] = rb_dict[player]['games'] +1

C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [70]:
temp = pd.concat(rb_dict)
temp.head()
    

games         Name Opponent Team  Week  att  def_rk  games  \
Aaron Jones 0      1  Aaron Jones      CHI   GB     4   13      11      2   
            1      2  Aaron Jones      DAL   GB     5   19       8      3   
            2      3  Aaron Jones      MIN   GB     6   13       1      4   
            3      4  Aaron Jones       NO   GB     7   17      17      5   
            4      5  Aaron Jones      DET   GB     9    5      31      6   

               h/a_1  points  rec_yds  surface_0  tar  td  yds  year  
Aaron Jones 0    1.0    10.9      NaN        1.0    0   1   49  2017  
            1    0.0    23.4      NaN        0.0    1   1  125  2017  
            2    0.0     5.2      NaN        0.0    4   0   41  2017  
            3    1.0    25.8      NaN        1.0    5   1  131  2017  
            4    1.0     3.1      NaN        1.0    5   0   12  2017

In [72]:
rb = temp.reset_index(drop=True)
rb.shape

(11896, 16)

In [15]:
wk8.rename(columns ={
                      'RushingAttempts':'att',
                      'RushingYards':'yds',
                      'RushingTouchdowns':'td',
                      'ReceivingTargets':'tar',
                      'ReceivingYards': 'rec_yds',
                    'FumblesLost':'fum',
                      'FantasyPointsDraftKings':'points'},inplace=True)
wk8.columns


Index(['Rank', 'Name', 'Team', 'Position', 'Week', 'Opponent', 'att', 'yds',
       'RushingYardsPerAttempt', 'td', 'tar', 'Receptions', 'rec_yds',
       'ReceivingTouchdowns', 'Fumbles', 'fum',
       'FantasyPointsPerGameDraftKings', 'points', 'def_rk'],
      dtype='object')

In [30]:
df.columns

Index(['games', 'year', 'Name', 'Opponent', 'Team', 'Week', 'att', 'yds', 'td',
       'tar', 'rec_yds', 'def_rk', 'h/a_1', 'surface_0', 'points'],
      dtype='object')

In [16]:
wk8.drop(['Receptions','RushingYardsPerAttempt', 'FantasyPointsPerGameDraftKings', 'Rank','Position', 'Fumbles','ReceivingTouchdowns','fum'],axis=1, inplace=True)
wk8.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'att', 'yds', 'td', 'tar',
       'rec_yds', 'points', 'def_rk'],
      dtype='object')

In [25]:
def_2018.head()

,Rank,Name,Team,Position,Played,DraftKingsQuarterbackFantasyPointsAllowedAverage,DraftKingsRunningbackFantasyPointsAllowedAverage,DraftKingsWideReceiverFantasyPointsAllowedAverage,DraftKingsTightEndFantasyPointsAllowedAverage,DraftKingsKickerFantasyPointsAllowedAverage,DraftKingsFantasyPointsAllowedAverage
0,1,Baltimore Ravens,BAL,DST,7,14.60,14.57,32.21,11.77,2.57,75.73
1,2,Jacksonville Jaguars,JAX,DST,7,15.37,20.50,29.46,8.50,7.14,80.97
2,3,Dallas Cowboys,DAL,DST,7,17.29,22.43,29.70,10.64,4.00,84.06
3,4,Seattle Seahawks,SEA,DST,6,15.17,22.97,35.78,8.25,3.00,85.17
4,5,Tennessee Titans,TEN,DST,7,17.78,17.80,41.56,8.04,4.43,89.61


In [17]:
wk8.head()

,Name,Team,Week,Opponent,att,yds,td,tar,rec_yds,points,def_rk
0,James Conner,PIT,8,CLE,24,146,2,6,66,41.2,28
1,Todd Gurley,LAR,8,GB,25,114,0,7,81,36.5,25
2,Marlon Mack,IND,8,OAK,25,132,2,4,17,31.9,27
3,Joe Mixon,CIN,8,TB,21,123,2,4,15,31.8,23
4,Adrian Peterson,WAS,8,NYG,26,149,1,1,7,30.6,24


In [62]:
df.shape

(13056, 15)

## Add Defensive Ranking

In [14]:
wk8 = add_defensive_rank(wk8, def_2018, 'rb')
wk8

,Rank,Name,Team,Position,Week,Opponent,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings,def_rk
0,1,James Conner,PIT,RB,8,CLE,24,146,6.1,2,6,5,66,0,1,0,41.2,41.2,28
1,2,Todd Gurley,LAR,RB,8,GB,25,114,4.6,0,7,6,81,1,0,0,36.5,36.5,25
2,3,Marlon Mack,IND,RB,8,OAK,25,132,5.3,2,4,2,17,0,0,0,31.9,31.9,27
3,4,Joe Mixon,CIN,RB,8,TB,21,123,5.9,2,4,3,15,0,0,0,31.8,31.8,23
4,5,Adrian Peterson,WAS,RB,8,NYG,26,149,5.7,1,1,1,7,1,1,1,30.6,30.6,24
5,6,Alvin Kamara,NO,RB,8,MIN,13,45,3.5,1,8,7,31,1,0,0,26.6,26.6,26
6,7,James White,NE,RB,8,BUF,8,15,1.9,1,13,10,79,0,0,0,25.4,25.4,17
7,8,Kenyan Drake,MIA,RB,8,HOU,12,58,4.8,1,2,2,37,1,0,0,23.5,23.5,20
8,9,Chris Carson,SEA,RB,8,DET,25,105,4.2,1,2,2,19,0,0,0,23.4,23.4,19
9,10,Lamar Miller,HOU,RB,8,MIA,18,133,7.4,1,2,0,0,0,0,0,22.3,22.3,18


In [18]:
loc = location[location['year']==2018]
loc = loc[loc['week']==8]
loc.shape

(14, 7)

## Home or Away

In [19]:
loc_wk = {}
home_lst = []
for week in loc['week'].unique():
    loc_wk[week]  = loc[loc['week']==week]
for idx, team in enumerate(wk8['Team']):
    if team in list(loc['home']):
        home_lst.append('home')           
    else:
        home_lst.append('away')
wk8['h/a'] = home_lst

In [21]:
wk8.head()

,Name,Team,Week,Opponent,att,yds,td,tar,rec_yds,points,def_rk,h/a
0,James Conner,PIT,8,CLE,24,146,2,6,66,41.2,28,home
1,Todd Gurley,LAR,8,GB,25,114,0,7,81,36.5,25,home
2,Marlon Mack,IND,8,OAK,25,132,2,4,17,31.9,27,away
3,Joe Mixon,CIN,8,TB,21,123,2,4,15,31.8,23,home
4,Adrian Peterson,WAS,8,NYG,26,149,1,1,7,30.6,24,away


## Surface

In [22]:
surface_dict={'KC':'grass',
             'CAR':'grass',
              'WAS':'grass',
              'CLE':'grass',
              'MIA':'grass',
              'PIT':'grass',
              'GB':'grass',
              'SF':'grass',
              'PHI':'grass',
              'LAR':'grass',
              'BAL': 'grass',
              'TEN':'grass',
              'OAK':'grass',
              'LAC':'grass',
              'TB':'grass',
              'CHI':'grass',
              'DEN':'grass',
              'ARI':'grass',
              'DAL':'turf',
              'SEA':'turf',
              'DET':'turf',
              'ATL':'turf',
              'NE':'turf',
              'IND':'turf',
              'NO':'turf',
              'NYG':'turf',
              'NYJ':'turf',
              'HOU':'turf',
              'BUF':'turf',
              'CIN':'turf',
              'MIN':'turf',
              'JAX':'grass'
              
             }

In [23]:
wk8.index = wk8['Team']

# Add the type of field the player palyed on by mapping the field on which the game was played to the surface dictionary

surface = []
for idx, team in enumerate(wk8['Team']):
    if wk8.iloc[idx]['h/a']=='home':
        surface.append(surface_dict[team])
    else:
        surface.append(surface_dict[wk8.iloc[idx]['Opponent']])


wk8['surface'] = surface

In [24]:
wk8.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'att', 'yds', 'td', 'tar',
       'rec_yds', 'points', 'def_rk', 'h/a', 'surface'],
      dtype='object')

In [25]:
wk8.head()

,Name,Team,Week,Opponent,att,yds,td,tar,rec_yds,points,def_rk,h/a,surface
Team,,,,,,,,,,,,,
PIT,James Conner,PIT,8,CLE,24,146,2,6,66,41.2,28,home,grass
LAR,Todd Gurley,LAR,8,GB,25,114,0,7,81,36.5,25,home,grass
IND,Marlon Mack,IND,8,OAK,25,132,2,4,17,31.9,27,away,grass
CIN,Joe Mixon,CIN,8,TB,21,123,2,4,15,31.8,23,home,turf
WAS,Adrian Peterson,WAS,8,NYG,26,149,1,1,7,30.6,24,away,turf


## Encode

In [26]:
wk8 = one_hot(wk8)

In [27]:
wk8.drop(['h/a_0','surface_1','surface','h/a','surface_encoded','h/a_encoded'], axis=1, inplace=True)
wk8.shape

(95, 13)

In [28]:
wk8.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'att', 'yds', 'td', 'tar',
       'rec_yds', 'points', 'def_rk', 'h/a_1', 'surface_0'],
      dtype='object')

In [29]:
wk8['year']=2018

In [55]:
rb.shape

(12405, 15)

In [56]:
rb.head()

,games,year,Name,Team,Week,Opponent,att,yds,td,tar,rec_yds,def_rk,h/a_1,surface_0,points
0,1,2012,Adrian Peterson,MIN,1,JAX,17,84,2,1,3,30,1.0,0.0,21.7
1,1,2012,Ahmad Bradshaw,NYG,1,DAL,17,78,1,2,15,16,1.0,0.0,17.3
2,1,2012,Alfred Morris,WAS,1,NO,28,96,2,0,0,29,0.0,0.0,21.6
3,1,2012,Andre Brown,NYG,1,DAL,0,0,0,0,0,16,1.0,0.0,0.0
4,1,2012,Anthony Allen,BAL,1,CIN,4,13,0,0,0,27,1.0,1.0,1.3


## Games

In [30]:
games_lst = []
temp = wk8.set_index('Name')
for name in temp.index:
    if name in list(df['Name'].unique()):
        games_lst.append(df[df['Name']==name]['games'].max()+1)
    else:
        games_lst.append(1)
    

In [31]:
wk8['games'] = games_lst

In [32]:
wk8

,Name,Team,Week,Opponent,att,yds,td,tar,rec_yds,points,def_rk,h/a_1,surface_0,year,games
0,James Conner,PIT,8,CLE,24,146,2,6,66,41.2,28,1.0,1.0,2018,7
1,Todd Gurley,LAR,8,GB,25,114,0,7,81,36.5,25,1.0,1.0,2018,21
2,Marlon Mack,IND,8,OAK,25,132,2,4,17,31.9,27,0.0,1.0,2018,4
3,Joe Mixon,CIN,8,TB,21,123,2,4,15,31.8,23,1.0,0.0,2018,6
4,Adrian Peterson,WAS,8,NYG,26,149,1,1,7,30.6,24,0.0,0.0,2018,54
5,Alvin Kamara,NO,8,MIN,13,45,1,8,31,26.6,26,0.0,0.0,2018,7
6,James White,NE,8,BUF,8,15,1,13,79,25.4,17,0.0,0.0,2018,31
7,Kenyan Drake,MIA,8,HOU,12,58,1,2,37,23.5,20,0.0,0.0,2018,8
8,Chris Carson,SEA,8,DET,25,105,1,2,19,23.4,19,0.0,0.0,2018,6
9,Lamar Miller,HOU,8,MIA,18,133,1,2,0,22.3,18,1.0,0.0,2018,98


## Reshape

In [58]:
wk7['year']=2018

In [38]:
cols = wk8.columns.tolist()
cols = cols[-1:] + cols[-2:-1] +cols[:9] + cols[-5:-2] + cols[-6:-5]
wk8 = wk8[cols]
wk8.head()

,games,year,Name,Team,Week,Opponent,att,yds,td,tar,rec_yds,def_rk,h/a_1,surface_0,points
0,7,2018,James Conner,PIT,8,CLE,24,146,2,6,66,28,1.0,1.0,41.2
1,21,2018,Todd Gurley,LAR,8,GB,25,114,0,7,81,25,1.0,1.0,36.5
2,4,2018,Marlon Mack,IND,8,OAK,25,132,2,4,17,27,0.0,1.0,31.9
3,6,2018,Joe Mixon,CIN,8,TB,21,123,2,4,15,23,1.0,0.0,31.8
4,54,2018,Adrian Peterson,WAS,8,NYG,26,149,1,1,7,24,0.0,0.0,30.6


In [37]:
df.columns

Index(['games', 'year', 'Name', 'Team', 'Week', 'Opponent', 'att', 'yds', 'td',
       'tar', 'rec_yds', 'def_rk', 'h/a_1', 'surface_0', 'points'],
      dtype='object')

## Concatenate

In [39]:
rb = pd.concat([df,wk8])
rb.shape

(12612, 15)

In [40]:
rb.tail()

,games,year,Name,Team,Week,Opponent,att,yds,td,tar,rec_yds,def_rk,h/a_1,surface_0,points
90,47,2018,Derrick Coleman,ARI,8,SF,0,0,0,0,0,5,1.0,1.0,0.0
91,2,2018,TJ Logan,ARI,8,SF,0,0,0,0,0,5,1.0,1.0,0.0
92,29,2018,Zach Line,NO,8,MIN,0,0,0,0,0,26,0.0,0.0,0.0
93,18,2018,Marcus Murphy,BUF,8,NE,0,0,0,0,0,9,1.0,0.0,0.0
94,8,2018,Wayne Gallman,NYG,8,WAS,1,-1,0,0,0,7,1.0,0.0,-0.1


In [41]:
rb.to_csv('input/rb/rb_loc_cleaned.csv', index=False)

In [10]:
# Parse upcoming week file by position

def clean_current(csv):# Read in files

    df_upcoming = pd.read_csv(csv)
    def_2018 = pd.read_csv('input/def/fantasy-football-points-allowed-defense-by-position.csv')

    # Obtain home and away attributes
    df_upcoming['year']=2018
    df_upcoming['home'] = df_upcoming['Game Info'].apply(lambda x: (x.split('@')[1]).split()[0])
    df_upcoming['away'] = df_upcoming['Game Info'].apply(lambda x: (x.split('@')[0]))

    rb = []
    wr = []
    te = []
    qb= []
    for idx,pos in enumerate(df_upcoming['Position']):
        if pos == 'RB':
            rb.append(idx)
        elif pos =='WR':
            wr.append(idx)
        elif pos =='TE':
            te.append(idx)
        elif pos =='QB':
            qb.append(idx)


    # Sort current week dataframe

    qb_df = df_upcoming.iloc[qb]
    rb_df = df_upcoming.iloc[rb]
    wr_df = df_upcoming.iloc[wr]
    te_df = df_upcoming.iloc[te]
    
    return qb_df,wr_df,te_df, rb_df

In [11]:
# Prepare defense dataframe by sorting by relevant attribute reseting the indexand using rank as the reset dataframe index +1

def add_defensive_rank(df, df1, position):
    df1.sort_values(position_dict[position], inplace=True)
    df1.reset_index(inplace=True, drop=True)
    df1['Rank']= df1.index+1
    df1.index=df1['Team']

# Map defensive rnking to eack player in the statist  ical dataframe 

    df['def_rk'] = list(map(lambda x: df1.loc[x]['Rank'], df['Opponent']))
    
    return df


In [12]:
# A function designed to add the Draftkings fantasy football salary to every players statistics dataframe

def add_salary(df,df2):
    df = df.sort_values(['Week','PlayerID'])
    df = df.drop(['FantasyPointsPerGameDraftKings','Rank','Position'], axis=1)
    df.reset_index(inplace=True, drop=True)
    df2 = df2.sort_values(['Week','PlayerID'])
    df2 = df2.drop(['Rank','Position'], axis=1)
    df2.reset_index(inplace=True, drop=True)
    df['salary'] = df2['DraftKingsSalary']
    
    return df

    

In [13]:
# One hot encode all non numeric attributes


def one_hot(df):
    df.reset_index(inplace=True, drop=True)
    encoder = LabelEncoder()
    df['h/a_encoded'] = encoder.fit_transform(df['h/a'])
    df['surface_encoded'] = encoder.fit_transform(df['surface'])


    home =OneHotEncoder()
    surface = OneHotEncoder()

    X = home.fit_transform(df['h/a_encoded'].values.reshape(-1,1)).toarray()
    Xs = surface.fit_transform(df['surface_encoded'].values.reshape(-1,1)).toarray()

    dfOneHot = pd.DataFrame(X, columns = ["h/a_"+str(int(i)) for i in range(X.shape[1])])
    df = pd.concat([df, dfOneHot], axis=1)


    dfOneHot = pd.DataFrame(Xs, columns = ['surface_'+str(int(i)) for i in range(Xs.shape[1])])
    df = pd.concat([df, dfOneHot], axis=1)

    return df

In [97]:
csv = 'input/qb/DKSalaries.csv'
qb7, wr7, te7, rb7 = clean_current(csv)
